# Word Generation In $G=\mathbb{Z}_2*\mathbb{Z}$

### Write $G=\langle x|x^2=1\rangle*\langle y \rangle$, with generating set $S=\{x,y,y^{-1}\}$.

Our interest is to generate all the **excursions** on the Cayley Graph of $G$ wrt $S$. These are words that end up back at one (Eg. $xx$ or $yy^{-1}$ or $yxxy^{-1}$). To avoid redundancy, we can generate **simple excursions**, those that don't go back to 1 "too early".

Let $Z_{g,X}$ be the set of words ($\in S^*$) that evaluates to $g\in G$ and contains no proper prefix in $X\subseteq G$.
Let $Z_{g,X}^{(n)}$ be such words of length $n$. In most cases, we only need $X\subseteq S\cup\{1\}$.

Our Goal is to find $Z_{1,\emptyset}^{(n)}$ for some small $n$. Contenations of such words of varied $n$ gives another such word. To avoid redundency, We shall compute $Z_{1,\{1\}}^{(n)}$, and notice that 

$$Z_{1,\emptyset}^{(n)}=\left(\cup_{\lambda}[\times_{i}Z_{1,\{1\}}^{(\lambda_i)}]\right)\cup Z_{1,\{1\}}^{(n)}$$

where $\lambda=(\lambda_i)$ ranges over the $2^{n-1}-1$ nontrivial compositions of $n$. Furthermore, the $2^{n-1}$ sets involved in the union expression are pairwise disjoint (Proof: Exercise for Jonah).


In [4]:
# Let 0,1,2 denote x,y,y^-1 respectively. Use e for the identity of G and the empty word

alph = [0,1,2] # Edit ONLY this line if you want to use different symbols

alph = list(map(str,alph))
nxt_alph_char = {alph[i]:alph[(i+1)] for i in range(len(alph)-1)}
nxt_alph_char[alph[-1]] = None
nxt_alph_char

{'0': '1', '1': '2', '2': None}

In [5]:
def char_inv(c):
    '''
    Recall: c in [0,1,2]
    Returns the inverse element of c
    '''
    return alph[(3-alph.index(c))%3]
    


class Word_Z2Z(object):
    def __init__(self,w=[]):
        self.w = []
        self.wred = [] # reduced word of w
        for c in w:
            self.add_char(c)
    
    def add_char(self,c):
        w = self.w
        wred = self.wred
        w.append(c)
        if self.wred[-1:]==[char_inv(c)]:
            wred.pop()
        else:
            wred.append(c)
    
    def del_last(self):
        # assume w is nonempty
        w = self.w
        c = w[-1]
        self.add_char(char_inv(c))
        w.pop()
        w.pop()
        return c
        
    def last(self):
        return self.w[-1]
        
    def word(self):
        return ''.join(map(str, self.w))
    
    def reduced_word(self):
        return ''.join(map(str, self.wred))
    
    def __len__(self):
        return len(self.w)
    
    def reduced_length(self):
        return len(self.wred)
        
    def __str__(self):
        return self.word()
    
    def __iter__(self):
        return iter(self.w)
    
    def clone(self):
        # bit more efficient than calling Word_Z2Z(self)
        other = Word_Z2Z()
        other.w = list(self.w)
        other.wred = list(self.wred)
        return other
    
    def clear(self):
        self.w = []
        self.wred = []
    

In [6]:
# initialize a new word generator
wgen = Word_Z2Z()

In [7]:
str(wgen)=='' and len(wgen)==0

True

In [8]:
from collections import defaultdict
# start simple. generate all words up to a given max length
maxlen = 5
wlist_by_len = defaultdict(list)
wd_reduction = {}

# depth first approach
#nit = 8

start = 1

while start or len(wgen):
    start = 0
    #print(wgen)
    wlist_by_len[len(wgen)].append(str(wgen))
    wd_reduction[str(wgen)] = wgen.reduced_word()
    if len(wgen)<maxlen:
        wgen.add_char(alph[0])
    else:
        while len(wgen) and nxt_alph_char[wgen.last()] is None:
            wgen.del_last()
        if len(wgen):
            wgen.add_char(nxt_alph_char[wgen.del_last()])
#     nit -=1
#     if not nit:
#         break

In [9]:
            
print(wlist_by_len)
print(wd_reduction)
len(wgen)==0        

defaultdict(<class 'list'>, {0: [''], 1: ['0', '1', '2'], 2: ['00', '01', '02', '10', '11', '12', '20', '21', '22'], 3: ['000', '001', '002', '010', '011', '012', '020', '021', '022', '100', '101', '102', '110', '111', '112', '120', '121', '122', '200', '201', '202', '210', '211', '212', '220', '221', '222'], 4: ['0000', '0001', '0002', '0010', '0011', '0012', '0020', '0021', '0022', '0100', '0101', '0102', '0110', '0111', '0112', '0120', '0121', '0122', '0200', '0201', '0202', '0210', '0211', '0212', '0220', '0221', '0222', '1000', '1001', '1002', '1010', '1011', '1012', '1020', '1021', '1022', '1100', '1101', '1102', '1110', '1111', '1112', '1120', '1121', '1122', '1200', '1201', '1202', '1210', '1211', '1212', '1220', '1221', '1222', '2000', '2001', '2002', '2010', '2011', '2012', '2020', '2021', '2022', '2100', '2101', '2102', '2110', '2111', '2112', '2120', '2121', '2122', '2200', '2201', '2202', '2210', '2211', '2212', '2220', '2221', '2222'], 5: ['00000', '00001', '00002', '0001

True

In [10]:
def is_reduced(w):
    for c1,c2 in zip(w[:-1],w[1:]):
        if c2==char_inv(c1):
            return False
    return True

# all the reduction should indeed be reduced
assert(all(map(is_reduced, wd_reduction.values())))

In [11]:
def gen_simple_excursions_up_to_length(n):
    wgen.clear()
    start = 1
    se_by_length = defaultdict(list)
    while start or len(wgen):
        if wgen.reduced_length()==0:
            # everything cancels. reduced word is the empty word, so we have an excursion
            se_by_length[len(wgen)].append(str(wgen))
            # for only simple excursions, must SKIP all the superstrings (Those with the current word as a proper prefix)
            # Note, in this case, the if statement below is not satisfied, so else block is entered.
            ## Thus, wgen becomes the next string in lex order that is at most the current length.
        if (start or wgen.reduced_length()) and len(wgen)<n and 2*wgen.reduced_length()<=n:
            # we don't count the empty word as a violating substring for simpleness.
            # The start variable helps make this exception.
            # if the reduced length is too long, we cannot "walk back in time" to form an excursion.
            wgen.add_char(alph[0])
        else:
            while len(wgen) and nxt_alph_char[wgen.last()] is None:
                wgen.del_last()
            if len(wgen):
                wgen.add_char(nxt_alph_char[wgen.del_last()])
        start = 0
    return se_by_length

In [267]:
import time

def time_test(func, params):
    runtimes = []
    for p in params:
        start = time.time()
        func(p)
        end = time.time()
        runtimes.append((p, end-start))
    return runtimes
    
    

In [13]:
runtimes = time_test(gen_simple_excursions_up_to_length, range(2,20,2))
runtimes

[(2, 0.0001578330993652344),
 (4, 0.0004637241363525391),
 (6, 0.003324508666992188),
 (8, 0.02486205101013184),
 (10, 0.2606596946716309),
 (12, 1.80329990386963),
 (14, 17.71326923370361),
 (16, 155.9225111007690),
 (18, 1505.366448163986)]

In [15]:
from datetime import timedelta
for n, dur in runtimes:
    print('%2d\t%7s'%(n, timedelta(seconds=float(dur))))

 2	0:00:00.000158
 4	0:00:00.000464
 6	0:00:00.003325
 8	0:00:00.024862
10	0:00:00.260660
12	0:00:01.803300
14	0:00:17.713269
16	0:02:35.922511
18	0:25:05.366448


In [21]:
maxlen = 20

In [269]:
se_by_length = gen_simple_excursions_up_to_length(maxlen)

In [22]:
for n, ws in sorted(se_by_length.items()):
    print (n, len(ws))
print(list([len(se_by_length[n]) for n in range(0,maxlen+1,2)]))

0 1
1 0
2 3
3 0
4 6
5 0
6 24
7 0
8 120
9 0
10 672
11 0
12 4032
13 0
14 25344
15 0
16 164736
17 0
18 1098240
19 0
20 7468032
[1, 3, 6, 24, 120, 672, 4032, 25344, 164736, 1098240, 7468032]


In [271]:
# make sure these excursions are really simple.

se_sets = {n:set(se_by_length[n]) for n in range(maxlen+1)}

for n, sn in sorted(se_sets.items()):
    for s in sn:
        pref = ''
        for x in s[:-1]:
            pref+=x
            assert(pref not in se_sets[len(pref)])





In [17]:
# save and load routines

def _preconvert(save_mode):
    def _preconvert_word(w):
        if save_mode:
            return 'e' if not w else w
        return '' if w=='e' else w
    return _preconvert_word

def save_gen(fname, wlist, maxlen=None):
    if maxlen is None:
        maxlen = max([n for n in wlist if wlist[n]])
    with open(fname,'w') as handle:
        for n in range(maxlen+1):
            handle.write(' '.join(map(_preconvert(1),[str(n)]+wlist[n]))+'\n')

def load_gen(fname, skip_empty = 0):
    wlist = {}
    with open(fname) as handle:
        for line in handle:
            line = line.strip()
            if line:
                wn = line.split()
                n = ZZ(wn.pop(0))
                if skip_empty and not wn:
                    continue
                wlist[n] = list(map(_preconvert(0), wn))
    return defaultdict(list, wlist)
            

In [273]:
save_gen('se.txt',se_by_length)

In [274]:
test = load_gen('se.txt')
test == se_by_length


True

In [18]:
se_by_length = load_gen('se.txt')

In [276]:
var('y z t')

(y, z, t)

Let $E(t),S(t)$ be the series for $Z_{1,\emptyset},Z_{1,\{1\}}$, respectively. 
It is already determined that $$E(t)=-\frac{3 \, \sqrt{-8 \, t^{2} + 1} - 1}{2 \, {\left(9 \, t^{2} - 1\right)}}.$$

Clearly, $Z_{1,\emptyset}=\text{SEQ}(Z_{1,\{1\}}\setminus \{\epsilon\})$. Thus, $$E(t)=\frac{1}{1-(S(t)-1)}.$$

Rearranging, we get $S(t)=2-E(t)^{-1}$. We check the coefficients of Taylor series expansion for $S(t)$ at $0$.

In [277]:
E = -1/2*(1-3*sqrt(1-8*t^2))/(1-9*t^2)
pretty_print(E)
latex(E)

-1/2*(3*sqrt(-8*t^2 + 1) - 1)/(9*t^2 - 1)

-\frac{3 \, \sqrt{-8 \, t^{2} + 1} - 1}{2 \, {\left(9 \, t^{2} - 1\right)}}

In [278]:
S = 2-1/E
pretty_print(S)

2*(9*t^2 - 1)/(3*sqrt(-8*t^2 + 1) - 1) + 2

In [279]:

pretty_print(E.taylor(t, 0, maxlen))

8099511*t^18 + 1143999*t^16 + 163719*t^14 + 23823*t^12 + 3543*t^10 + 543*t^8 + 87*t^6 + 15*t^4 + 3*t^2 + 1

In [280]:

pretty_print(S.taylor(t, 0, maxlen))

1098240*t^18 + 164736*t^16 + 25344*t^14 + 4032*t^12 + 672*t^10 + 120*t^8 + 24*t^6 + 6*t^4 + 3*t^2 + 1